In [1]:
# Importa pandas, sqlite3 y stanza
import pandas as pd
import sqlite3
# Importing TextBlob
from textblob import TextBlob
# Importing the NaiveBayesAnalyzer classifier from NLTK
from textblob.sentiments import NaiveBayesAnalyzer
# # Importing seaborn
# import seaborn as sns


In [2]:
# Conexión a la base de datos SQLite
con = sqlite3.connect("rrss ampliado.db")

# Se usa read_sql_query de pandas para extraer el resultado de la consulta a un DataFrame
df = pd.read_sql_query("SELECT id_post,contenido FROM PRINCIPAL;", con)

# Cierre de la conexión
con.close()

df.head()
df


,id_post,contenido
0,rt7d711,Cryptid coins can be used to summon a Cryptid ...
1,rta7all,FF14 soundtracks officially uploaded to Youtub...
2,rtcgpme,Ukraine says it has spent the nearly $100 mill...
3,rt7aiac,Ukraine Spends $15M of Donated Crypto on Milit...
4,rt6jk23,"Coinbase CEO: ""we don’t think there’s a high r..."
...,...,...
43970,t1508732822663618564,I HAVE NEVER BEEN MORE BULLISH ON #BITCOIN!
43971,t1508755629158109185,One weekend rally and #Bitcoin is higher then ...
43972,t1514350075669065729,Don’t wait to buy #Bitcoin..\nBuy #Bitcoin & w...
43973,t1514628345014759433,#BITCOIN CHANGED MY LIFE!!!!!


In [3]:
import textblob
import re, emoji

tb = textblob.Blobber(analyzer=NaiveBayesAnalyzer())
df1=pd.DataFrame(columns=['palabras','etiquetas'])
lista_indice=[]
lista_etiqueta=[]
lista_palabra=[]
for index, row in df.iterrows():
    texto=emoji.get_emoji_regexp().sub("",row['contenido'])
    texto=re.sub(r'http\S+', '',texto)
    sent = tb(texto).sentiment
    df.loc[index, 'clasificación'] = sent[0]
    df.loc[index, 'positivo'] = sent[1]
    df.loc[index, 'negativo'] = sent[2]
    polaridad = tb(texto).polarity
    df.loc[index, 'polaridad'] = polaridad
    subjetividad = tb(texto).subjectivity
    df.loc[index, 'subjetividad'] = subjetividad
    tags = tb(texto).tags
    indice=df.loc[index,'id_post']
    ind=[indice]*len(tags)
    
    df2=pd.DataFrame(tags,index=ind,columns=['palabras','etiquetas'])
    df1=pd.concat([df1,df2])

df

C:\Users\emman\AppData\Local\Temp/ipykernel_30940/2784804719.py:10: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  texto=emoji.get_emoji_regexp().sub("",row['contenido'])


,id_post,contenido,clasificación,positivo,negativo,polaridad,subjetividad
0,rt7d711,Cryptid coins can be used to summon a Cryptid ...,pos,0.930178,0.069822,0.000000,0.000000
1,rta7all,FF14 soundtracks officially uploaded to Youtub...,pos,0.907206,0.092794,-0.125000,0.375000
2,rtcgpme,Ukraine says it has spent the nearly $100 mill...,neg,0.168787,0.831213,0.166667,0.333333
3,rt7aiac,Ukraine Spends $15M of Donated Crypto on Milit...,neg,0.499715,0.500285,-0.100000,0.100000
4,rt6jk23,"Coinbase CEO: ""we don’t think there’s a high r...",pos,0.554185,0.445815,0.107000,0.383000
...,...,...,...,...,...,...,...
43970,t1508732822663618564,I HAVE NEVER BEEN MORE BULLISH ON #BITCOIN!,neg,0.480751,0.519249,0.625000,0.500000
43971,t1508755629158109185,One weekend rally and #Bitcoin is higher then ...,pos,0.818483,0.181517,0.305556,0.466667
43972,t1514350075669065729,Don’t wait to buy #Bitcoin..\nBuy #Bitcoin & w...,neg,0.458584,0.541416,0.000000,0.000000
43973,t1514628345014759433,#BITCOIN CHANGED MY LIFE!!!!!,pos,0.602019,0.397981,0.000000,0.000000


In [5]:
###  INSERTA DATAFRAME EN LA BASE DE DATOS ####
import sqlite3
conexion= sqlite3.connect('rrss ampliado.db')
#Creamos el cursor.
cursor = conexion.cursor()

#Insertamos el dataframe a su correspondiente tabla.

#Tabla PLN
df.to_sql("PNL",con=conexion, if_exists='replace', index=False)

df1.to_sql("PALABRAS", con=conexion, if_exists='replace')

# conexion.execute("SELECT * FROM PNL").fetchall()
conexion.commit()

#cerramos la conexion 
conexion.close()